In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
import re
import random
import textwrap
%run ../py/sql_utlis.py

ZMQInteractiveShell_obj = get_ipython()
su = SqlUtilities()
_, CURSOR = su.get_jh_conn_cursor()

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]Invalid connection. (14) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (ParseConnectParams()). (14)')

In [7]:

# Get the incomplete child strings labeled in the minimum requirements column
sql_str = '''
    SELECT *
    FROM
        [Jobhunting].[dbo].[FileNames] fn INNER JOIN
        [Jobhunting].[dbo].[NavigableParentSequence] nps ON
        fn.[file_name_id] = nps.[file_name_id] INNER JOIN
        [Jobhunting].[dbo].[NavigableParents] np ON
        nps.[navigable_parent_id] = np.[navigable_parent_id];'''
nps_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
mask_series = (nps_df.mrs_id != 0)
file_name_ids_list = nps_df[mask_series].file_name_id.unique().tolist()
mask_series = nps_df.file_name_id.isin(file_name_ids_list)
CHILD_STRS_LIST = nps_df[mask_series].navigable_parent.tolist()

In [32]:

# Get all possible unlabeled HTML strings from the database
SQL_STR = '''
    SELECT
        np.[navigable_parent],
        mrs.[mrs_explanation]
    FROM
        [Jobhunting].[dbo].[NavigableParents] np INNER JOIN
        [Jobhunting].[dbo].[NavigableParentSequence] nps ON
        nps.[navigable_parent_id] = np.[navigable_parent_id] INNER JOIN
        [Jobhunting].[dbo].[MinimumRequirementsSection] mrs ON
        nps.[mrs_id] = mrs.[mrs_id]
    WHERE
        (np.[is_task_scope] IS NULL) OR
        (np.[is_minimum_qualification] IS NULL) OR
        (np.[is_preferred_qualification] IS NULL) OR
        (np.[is_legal_notification] IS NULL) OR
        (np.[is_job_title] IS NULL) OR
        (np.[is_office_location] IS NULL) OR
        (np.[is_job_duration] IS NULL) OR
        (np.[is_supplemental_pay] IS NULL) OR
        (np.[is_educational_requirement] IS NULL) OR
        (np.[is_interview_procedure] IS NULL) OR
        (np.[is_corporate_scope] IS NULL) OR
        (np.[is_posting_date] IS NULL) OR
        (np.[is_other] IS NULL);'''

In [363]:

def get_sql_code():
    nonheader_quals_df = pd.DataFrame(su.get_execution_results(CURSOR, SQL_STR, verbose=False))
    mask_series = nonheader_quals_df.navigable_parent.isin(CHILD_STRS_LIST)
    output_str = ''
    df = nonheader_quals_df[mask_series]
    records_count = df.shape[0]
    output_str += f'\n# Only {records_count:,} records to go\n'
    if records_count:
        df = df.sample(1)
        output_str += f"# This tag is labeled as {df.mrs_explanation.squeeze()}\ntag_str =  '"
        tag_str = df.navigable_parent.squeeze().replace("'", "\\'")
        output_str += "'\ntag_str += '".join(textwrap.wrap(tag_str, expand_tabs=False,
                                                           replace_whitespace=False,
                                                           break_long_words=False, drop_whitespace=False))
        output_str += "'\n"
        output_str += 'sql_str = r"""UPDATE NavigableParents\n'
        output_str += '    SET\n        is_header = 0, is_task_scope = 0,\n'
        output_str += '        is_minimum_qualification = 0, is_preferred_qualification = 0,\n'
        output_str += '        is_legal_notification = 0, is_job_title = 0,\n'
        output_str += '        is_office_location = 0, is_job_duration = 0,\n'
        output_str += '        is_supplemental_pay = 0, is_educational_requirement = 0,\n'
        output_str += '        is_interview_procedure = 0, is_corporate_scope = 0,\n'
        output_str += '        is_posting_date = 0, is_other = 0\n'
        output_str += '    WHERE (navigable_parent LIKE ? ESCAPE \'\\\')"""\n'
        output_str += r"cursor_obj = CURSOR.execute(sql_str, (su.wc_rgx.sub(r'\\\g<1>', tag_str),))"
        output_str += '\nCURSOR.commit()'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_sql_code(), replace=True)